Scraping Github Repos from multiple pages

In [1]:
import requests
import pandas as pd
import time

/tmp/ipykernel_570/4032102518.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [34]:
project_name = []
last_update = []

In [35]:
def get_projects(stars_range):
    i = 1;
    while i <= 100:
        
        headers = {
            'authority': 'github.com',
            'accept': 'application/json',
            'accept-language': 'en-US,en;q=0.9',
            'referer': 'https://github.com/search?q=stars%3A'+str(stars_range)+'&type=repositories&p=1',
            'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
            'x-github-target': 'dotcom',
            'x-requested-with': 'XMLHttpRequest',
        }

        params = {
            'q': 'stars:'+str(stars_range),
            'type': 'repositories',
            'p': str(i),
        }
        
        try:
            # response variable
            response = requests.get('https://github.com/search', params=params, headers=headers) 
            response.raise_for_status()
            print(f"Page {i} - Status Code: {response.status_code}")
            
            # json object
            results_json = response.json()
            
        except requests.exceptions.RequestException as e:
            print(f"Request failed on page {i}: {e}")
            delay_seconds = 60  # default delay
            time.sleep(delay_seconds)
            continue
            
        except json.decoder.JSONDecodeError as e:
            print(f"JSONDecodeError on page {i}: {e}")
        
        project_list = results_json['payload']['results']
        if not project_list:
            break
        
        for project in project_list:
            project_name.append(project['hl_name'])
            last_update.append(project['repo']['repository']['updated_at'])
        
        i += 1


In [36]:
get_projects('80000..100000')

Page 1 - Status Code: 200
Page 2 - Status Code: 200
Page 3 - Status Code: 200
Page 4 - Status Code: 200


In [37]:
projects_df = pd.DataFrame({'Project':project_name, 'Last Update':last_update})

In [38]:
projects_df

,Project,Last Update
0,f/awesome-chatgpt-prompts,2024-02-09T02:04:13.449Z
1,Genymobile/scrcpy,2024-02-17T15:15:36.630Z
2,mrdoob/three.js,2024-02-17T20:56:14.164Z
3,microsoft/TypeScript,2024-02-17T10:26:22.428Z
4,goldbergyoni/nodebestpractices,2024-02-11T22:58:52.398Z
5,angular/angular,2024-02-17T21:37:49.342Z
6,microsoft/terminal,2024-02-17T16:32:39.411Z
7,denoland/deno,2024-02-17T22:22:47.093Z
8,mui/material-ui,2024-02-17T14:37:42.594Z
9,rust-lang/rust,2024-02-17T22:30:09.305Z


In [7]:
#projects_df.to_excel("projects_test.xlsx", index=False)

try:
    with pd.ExcelWriter(
        "projects_9500AndUp.xlsx",
        mode="a",
        engine="openpyxl",
        if_sheet_exists="overlay",
    ) as writer:
         projects_df.to_excel(writer,sheet_name="Sheet1", startrow=writer.sheets["Sheet1"].max_row, index = False,header= False)
except FileNotFoundError:
    projects_df.to_excel("projects_9500AndUp.xlsx", index=False)